# 카카오톡 감정 분석 함수

### 카카오톡 파일 처리 conversation_to_list

In [98]:
import os
os.chdir("C:/Users/beomj/Onedrive/바탕 화면");

        
def conversation_to_list(kakaotalk_file_name):
    
    name_list = []
    talk_list = []
    
    f = open(kakaotalk_file_name,'rt',encoding='UTF8');
    lines = f.readlines()
    for line in lines:
        if(line.find('[')==0):
            index1 = line.find('[') +1
            index2 = line.find(']')
            index3 = line.find(']',index2+1) +1

            name2 = line[index1:index2]
            name_list.append(name2)
            talk_list.append(line[index3+1:])

    f.close()

    return name_list, talk_list
        
name_list, talk_list=conversation_to_list("3355.txt")

박범진
안녕하세요

박범진
톡게시판 '투표': 조이름

권현우
책 새거 25000에 중고로 파는데 이거 사서 제본하는거 어때요?



### 파파고 번역 conv_translate

In [85]:
# 파파고 스트링 번역
def papago_translate(context):
    try:
        url = "https://openapi.naver.com/v1/papago/n2mt"
        headers= {"X-Naver-Client-Id": "svrfjDI6SLeRFekWQEOz", "X-Naver-Client-Secret":"JLKglR7QQM"}
        params = {"source": "ko", "target": "en", "text": context}
        response = requests.post(url, headers=headers, data=params)
        res = response.json()
        return res['message']['result']['translatedText']
    except:
        return "번역 실패"

# talk_list 번역
def conv_translate(talk_list):
    translated_list = []
    for i in talk_list:
        new = papago_translate(i)
        translated_list.append(new)
    return translated_list


In [116]:
trans_list = conv_translate(talk_list[0:20])

In [ ]:
trans_list